<a href="https://colab.research.google.com/github/Djensonsan/Information_Retrieval_Assignment_2/blob/main/LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Information Retrieval Assignment 2: LDA


## Runtime specs

In [1]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 85
model name	: Intel(R) Xeon(R) CPU @ 2.00GHz
stepping	: 3
microcode	: 0x1
cpu MHz		: 2000.186
cache size	: 39424 KB
physical id	: 0
siblings	: 4
core id		: 0
cpu cores	: 2
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm mpx avx512f avx512dq rdseed adx smap clflushopt clwb avx512cd avx512bw avx512vl xsaveopt xsavec xgetbv1 xsaves arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	:

In [2]:
!cat /proc/meminfo

MemTotal:       26751732 kB
MemFree:        21632444 kB
MemAvailable:   25739404 kB
Buffers:           99444 kB
Cached:          4197412 kB
SwapCached:            0 kB
Active:          1356236 kB
Inactive:        3388708 kB
Active(anon):     405960 kB
Inactive(anon):      404 kB
Active(file):     950276 kB
Inactive(file):  3388304 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:              1012 kB
Writeback:             0 kB
AnonPages:        448076 kB
Mapped:           242644 kB
Shmem:               940 kB
Slab:             240084 kB
SReclaimable:     182336 kB
SUnreclaim:        57748 kB
KernelStack:        4976 kB
PageTables:         6200 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    13375864 kB
Committed_AS:    3220564 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
Percpu:             1904 kB
AnonHugePages:   

## Imports

In [3]:
# Install your required packages here
!pip install pandas numpy matplotlib fsspec gcsfs dask
!pip install -q tqdm

In [4]:
import pandas as pd
import numpy as np
import tqdm.notebook as tqdm
import dask.dataframe as dd

import gensim
from tqdm import tqdm
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.corpus import stopwords
import nltk
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# Mount google drive in colab:
from google.cloud import storage
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Preprocessing

In [6]:
# Pycharm:
# data = pd.read_csv('data/news_dataset.csv')

In [7]:
# Colab:
data = pd.read_csv('/content/drive/MyDrive/IR-Assignment-2/data/news_dataset.csv')

### Exploration

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141585 entries, 0 to 141584
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   id           141585 non-null  int64 
 1   title        141583 non-null  object
 2   publication  141564 non-null  object
 3   author       125725 non-null  object
 4   date         138869 non-null  object
 5   year         138911 non-null  object
 6   month        138911 non-null  object
 7   url          84541 non-null   object
 8   content      141543 non-null  object
dtypes: int64(1), object(8)
memory usage: 9.7+ MB


In [9]:
data.head(n=3)

,id,title,publication,author,date,year,month,url,content
0,1,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016,12,NaN,WASHINGTON — Congressional Republicans have...
1,2,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017,6,NaN,"After the bullet shells get counted, the blood..."
2,3,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017,1,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri..."


### Keep document content

In [10]:
data_content = data['content']

In [11]:
type(data_content)

pandas.core.series.Series

In [12]:
data_content.head(n=3)

0    WASHINGTON  —   Congressional Republicans have...
1    After the bullet shells get counted, the blood...
2    When Walt Disney’s “Bambi” opened in 1942, cri...
Name: content, dtype: object

### Tokenization, Stemming and Lemmatization

In [13]:
tqdm.pandas()
# Note Jens: Might want to use Dask to speed things up. 
# When using Dask can't use tqdm as far as I know.

In [14]:
# There's NaN values in the dataset
data_content.dropna(inplace=True)

In [15]:
data_content.isna().any()

False

In [16]:
# Tokenization
data_content_tokenized = data_content.progress_apply(lambda x: nltk.word_tokenize(x))

100%|██████████| 141543/141543 [14:52<00:00, 158.56it/s]


In [17]:
# Remove words smaller than 3 characters
data_content_tokenized = data_content_tokenized.progress_apply(lambda x: [y for y in x if len(y)>2])

100%|██████████| 141543/141543 [00:19<00:00, 7158.94it/s]


In [18]:
# Stemming and Lemmatization 
stemmer = SnowballStemmer("english")
data_content_stemmed = data_content_tokenized.progress_apply(lambda x: [stemmer.stem(WordNetLemmatizer().lemmatize(y)) for y in x])

100%|██████████| 141543/141543 [25:06<00:00, 93.98it/s]


In [19]:
# Remove Stopswords
stop_words = set(stopwords.words('english')) 
data_content_clean = data_content_stemmed.progress_apply(lambda x: [y for y in x if not y in stop_words])

100%|██████████| 141543/141543 [00:16<00:00, 8671.00it/s]


In [20]:
data_content_clean.head()

0    [washington, congression, republican, new, fea...
1    [bullet, shell, get, count, blood, dri, votiv,...
2    [walt, disney, bambi, open, 1942, critic, prai...
3    [death, may, great, equal, necessarili, evenha...
4    [seoul, south, korea, north, korea, leader, ki...
Name: content, dtype: object

In [21]:
data_content_clean.to_csv('/content/drive/MyDrive/IR-Assignment-2/data/new_dataset_clean.csv')

## LDA Library

## Topic Generation

## Experimentation

## Evaluation

## Top Documents